# Open-Source Software PiTCT for Supervisory Control Design

## Part 1: Basic Operations in PiTCT

In this notebook, we demonstrate the functionality of PiTCT for basic automaton operations.  
We will follow three main steps:
1.  **Modeling**: Define a discrete-event system as an automaton.
2.  **Analysis**: Analyze key properties of automaton: reachability, coreachability, nonblocking, and trim.
3.  **Synchronous Product**: Compute product of two automata while synchronizing on their shared events.

## Printer Example

We create an automaton named `PRINTER` that models the high-level behavior of a simple printer. The system has 4 states:

- **State 0 (IDLE)**: The printer is ready and waiting.
- **State 1 (WORKING)**: The printer is currently printing.
- **State 2 (BROKEN)**: The printer has malfunctioned and needs repair.
- **State 3 (JUNK)**: The printer is dumped.

State transitions are triggered by event occurrences:

- **start**: Start a printing job, transitioning from IDLE to WORKING.
- **auto_finish**: Complete a printing job successfully, transitioning from WORKING to IDLE.
- **manual_stop**: Stop a printing job manually, transitioning from WORKING to IDLE.
- **breakdown**: Break down (e.g. paper jam), transitioning from WORKING to BROKEN.
- **fix**: Fix the breakdown problem, transitioning from BROKEN to IDLE.
- **dump**: Dump the broken printer, transitioning from BROKEN to JUNK.

**Initial State**: States 0 (IDLE)

**Marker States**: States 0 (IDLE) and 1 (WORKING) are marked as "acceptable" states.  

## Install PiTCT

PiTCT is distributed as a Python package via PyPI. To start using it, we need to install it into our current Python environment. The command below uses pip (Python's package installer) to download and install the library.

In [5]:
%pip install pitct

### 0. Initialization
Before performing any operations, we must initialize the PiTCT environment.

`pitct.init('printer', overwrite=True)` does two things:
1.  Creates a folder named **`printer`** in your current directory.
2.  Sets it as the workspace. All generated automaton files (e.g., `PRINTER.DES`) will be saved inside this folder.

In [6]:
import pitct
pitct.init('printer', overwrite=True)

### 1. Modeling

In PiTCT, we define an automaton by specifying three key components: **State Number**, **State Transitions**, and **Marker States**.

**State Number**: Given a state number (`statenum = n`), PiTCT automatically labels them with integers from `0` to `n-1`.
- **State 0**: Always set to be the **Initial State**.
- **State 0, ..., n-1**: Used as specific IDs in state transitions and marker states below.

**State Transitions**: Transitions are defined as a list of triples: `(exit_state, event, enter_state)`.
- **Events**: You can use either **strings** (e.g., `'start'`) or **natural numbers** (e.g., `10`, `20`). However, you **cannot mix** strings and numbers in the same model.
- **Default Controllability**: By default, all events defined this way are treated as **controllable ('c')**.  
  (Uncontrollable events will be discussed in Part 2 in "2_supervisory_control_computation.ipynb").

**Marker States**: Marker states are defined as a list of states.  

In [7]:
# number of states
# states are sequentially labeled 0,1,...,statenum-1
# initial state is labeled 0
statenum = 4

# Transitions: (exit_state, event_label, enter_state)
trans = [
    (0, 'start', 1),
    (1, 'auto_finish', 0),
    (1, 'manual_stop', 0),
    (1, 'breakdown', 2),
    (2, 'fix', 0),
    (2, 'dump', 3),
]
# Marker states (good states)
marker = [0, 1]

pitct.create('PRINTER', statenum, trans, marker)

# visualization
pitct.display_automaton("PRINTER")

/lib/python3.13/site-packages/pitct/name_converter.py:70: UserWarning: controllable and uncontrollable status of event is not specified. All events are set to be controllable.
  warnings.warn("controllable and uncontrollable status of event is not specified. All events are set to be controllable.")


#### Inspecting the Model

Instead of plotting the automaton, we can inspect the created automaton using the following 4 helper functions. This is useful for debugging or verifying the automaton structure.

- `statenum`: Returns the number of states.
- `events`: Returns the list of all events.
- `trans`: Returns the list of all transitions. Note that 'c' (controllable) is attached by default to events.
- `marker`: Returns the list of marker states.

In [11]:
pitct.statenum("PRINTER")

4

In [12]:
pitct.events("PRINTER")

['auto_finish', 'manual_stop', 'breakdown', 'fix', 'dump', 'start']

In [13]:
pitct.trans("PRINTER")

[(0, 'start', 1, 'c'),
 (1, 'auto_finish', 0, 'c'),
 (1, 'manual_stop', 0, 'c'),
 (1, 'breakdown', 2, 'c'),
 (2, 'fix', 0, 'c'),
 (2, 'dump', 3, 'c')]

In [14]:
pitct.marker("PRINTER")

[0, 1]

### 2. Analysis
Once the automaton model is created, we can analyze its fundamental properties.

#### reachable

**Reachability** is the property that a state (or all states) can be accessed from the initial state via a sequence of events.
If a state is reachable, PiTCT can also generate a shortest string (sequence of events) to get there from the initial state.

In [15]:
# Check if all states are reachable
pitct.is_reachable("PRINTER")

True

In [16]:
# Check reachability of state 3
pitct.is_reachable("PRINTER", 3)

True

In [17]:
# get shortest path
pitct.reachable_string("PRINTER", 3)

['start', 'breakdown', 'dump']

#### coreachable

**Coreachability** is the property that a marker state can be reached from a given state.
If a state is **not** coreachable, it means that once the system enters this state, it can never reach a marker state.

In [18]:
# Check if all states are coreachable
pitct.is_coreachable("PRINTER")

False

In [19]:
# Check coreachability of state 2
pitct.is_coreachable("PRINTER", 2)

True

In [20]:
# Check coreachability of state 3
pitct.is_coreachable("PRINTER", 3)

False

In [21]:
# get shortest path
pitct.coreachable_string("PRINTER", 2)

['fix']

#### shortest_string

Sometimes it is convenient to get a shortest path from an arbitrary state (not necessarily the initial state) to another state.
For example, let's find the shortest path from State 1 (WORKING) to State 3 (JUNK).

In [22]:
pitct.shortest_string('PRINTER', 1, 3)

['breakdown', 'dump']

#### nonblocking

A system is **nonblocking** if every reachable state is also coreachable.
In other words, from any state that the system can reach, there is always a path to a marker state. If there are reachable states that are not coreachable (like the `JUNK` state in our example), the system is **blocking**.

In [23]:
pitct.is_nonblocking("PRINTER")

False

In [24]:
# checking blocking state
pitct.blocking_states("PRINTER")

[3]

#### trim

Since our analysis showed that the system is blocking (due to State 3), we want to create a "clean" version of the model. The trim operation removes all blocking states (states that cannot reach a marker) ass well as all nonreachable states.

This creates a new automaton `PRINTER_TRIM` that only contains the reachable and coreachable parts of the system. Note that the dump transition and State 3 are removed.

In [25]:
# check if PRINTER is trim or not
pitct.is_trim("PRINTER")

False

In [26]:
pitct.trim('PRINTER_TRIM', 'PRINTER')

print("Original states:", pitct.statenum('PRINTER'))
print("Trimmed states: ", pitct.statenum('PRINTER_TRIM'))

pitct.display_automaton('PRINTER_TRIM')

Original states: 4
Trimmed states:  3


In [27]:
# check if PRINTER_TRIM is trim or not
pitct.is_trim("PRINTER_TRIM")

True

### 3. Synchronous Product

Systems rarely operate in isolation. We now model a `USER` who interacts with the printer.  
The User can:

- Press **start** or **manual_stop**.
- **take_printouts** after an **auto_finish**.

To analyze the combined behavior, we compute the **Synchronous Product**.
The interaction is determined by **Shared Events**:

- **Shared Events** (Synchronized): `start`, `manual_stop`, `auto_finish`
  (Both systems must agree to execute these events.)
- **Private Events** (PRINTER only): `breakdown`, `fix`, `dump`
  (The Printer executes these events independently of the User.)
- **Private Events** (USER only): `take_printouts`
  (The User executes this event independently.)

In [28]:
# Define USER model
user_trans = [
    (0,'start',0), (0,'auto_finish',1), 
    (0,'manual_stop',0), (1,'take_printouts', 0)
]
pitct.create('USER', 2, user_trans, [0])
pitct.display_automaton('USER')

In [22]:
# sync operation
pitct.sync("PU","PRINTER","USER")
pitct.display_automaton("PU")

The `sync` operation creates a new global state set (cartesian product of the component state sets). The states in the new automaton `PU` are renumbered sequentially starting from 0.

To map these new state numbers back to the original state pairs `(PRINTER_state, USER_state)`, we use the `table=True` option.
The output format `k: i,j` means:
- **State k** in `PU` corresponds to:
- **State i** in `PRINTER` AND **State j** in `USER`.

In [29]:
table = pitct.sync("PU", "PRINTER", "USER", table=True, convert=True)
print(table)

pitct.display_automaton("PU")

0: 0,0
1: 1,0
2: 0,1
3: 2,0
4: 3,0

